This file contains random tests used throughout my research. Since I was told to not upload my code for my actual research, this file contains the code closest to the code I used in my research. I did most of my code and testing outside of this notebook, however.

Most notably in this file, I preprocessed a lot of data, coded the HMM2Vec algorithm, and ran a few tests that did not require my modified river library on here.

# set up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install river

In [ ]:
!pip3 install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
import sklearn.metrics as skm
from sklearn.utils import shuffle
import datetime
import time

#rom gensim.models import Word2Vec


from scipy.spatial.distance import euclidean

In [ ]:
from river import forest
from river import metrics
from river import evaluate
from river import stream
from river import neighbors
from river import drift
from river import ensemble
from river import compat
from river import tree

#from hmmlearn.hmm import CategoricalHMM

# load data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/all_families_w_opcodes.csv')

In [ ]:
df = df.drop(['Unnamed: 0'], axis = 1)

# import data + preprocess (DON'T RUN)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/v077_timestamps.txt', header = None)
df.rename(columns={0: 'file_name', 1: 'date'}, inplace=True)
df.drop_duplicates(subset='file_name', inplace=True)
df.dropna(subset=['date'], inplace=True)

In [ ]:
labels = pd.read_csv('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/00077.csv')

In [ ]:
df = df.merge(labels, on='file_name')

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['date'].sort_values(ascending = False).tail(20)

13287   1970-01-01
36196   1970-01-01
67112   1970-01-01
16123   1970-01-01
60482   1970-01-01
1561    1970-01-01
77979   1970-01-01
58188   1970-01-01
47863   1970-01-01
18214   1970-01-01
14127   1970-01-01
65278   1970-01-01
77987   1970-01-01
18205   1970-01-01
47852   1970-01-01
22979   1970-01-01
3458    1970-01-01
71033   1970-01-01
71034   1970-01-01
9349    1970-01-01
Name: date, dtype: datetime64[ns]

In [ ]:
# remove nonsensical dates + dates w/ very few data points per month
df = df[(df['date'] < '2013-07-01 00:00:00') & (df['date'] >= '2001-01-01 00:00:00')]  # data set is from 2018 (DOUBLE CHECK)

In [ ]:
fig2 = px.histogram(df, x = 'date', nbins = 100)
fig2.show()

In [ ]:
df.groupby('name').count().sort_values(by='file_name', ascending=False).head(15)

In [ ]:
df = df.filter(['file_name', 'name', 'date'])

In [ ]:
df = df.sort_values(by='date', ascending=True)

# get opcodes for all top families and merge (DON'T RUN)

In [ ]:
# filter families corresponding to masters paper
family_names = ["Allaple.A", "Beebone", "Diplugem", "Obfuscator", "OnLineGames", "Startpage", "Systex.A", "Vobfus", "Vundo", "Winwebsec", "Zbot"]

In [ ]:
opcodes = pd.DataFrame(columns=['file_name', 'text'])

for family in family_names:
  for filename in os.listdir('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/' + family):
    if filename.endswith('.txt'):
      path = '/content/drive/MyDrive/23-24/summer 24/adaptive learning research/' + family + '/' + filename
      with open(path, 'r') as file:
        text = file.read().replace('\n', ' ')
      opcodes.loc[int(len(opcodes.index))]  = [filename[:-4], text]

In [ ]:
df = df.merge(opcodes, on='file_name')

In [ ]:
df['name'].value_counts()

name
Vobfus         4030
Diplugem       2255
Zbot           1930
Vundo          1677
Obfuscator     1572
Winwebsec      1381
Beebone        1344
Startpage      1250
OnLineGames    1207
Systex.A       1098
Name: count, dtype: int64

In [ ]:
df

,file_name,name,date,text
0,VirusShare_1f0fb239c2d3156c05ab9277595c6ea0,Vobfus,2001-01-07 19:54:42,ljmp jae xchg push jae or out push jae add lea...
1,VirusShare_7a9477a52610e7a66428d2228f6c85fb,Vobfus,2001-01-09 20:15:00,ljmp jb add mov jb push jb add movsb jb mov mo...
2,VirusShare_3fa07ebbd74cbc1e537ddc9810dfcfb0,Zbot,2001-01-10 02:21:04,push mov sub push push push lea mov lea mov le...
3,VirusShare_2e4bdc2eb4ff94fc4fbcd3426196f830,Zbot,2001-01-10 03:03:21,push mov sub push push push lea mov lea mov le...
4,VirusShare_ad8a44ef537ac9b6c76dda4f848edc52,Zbot,2001-01-10 03:10:13,push mov sub push push push mov mov mov xor mo...
...,...,...,...,...
17739,VirusShare_23a52ac64efcc4e074fdd5b4230cf360,Beebone,2013-06-04 02:45:23,jmp jmp jmp jmp jmp jmp jmp jmp jmp jmp jmp jm...
17740,VirusShare_362a9b26ff18823258e16164f25d55b0,Beebone,2013-06-04 02:45:23,jmp jmp jmp jmp jmp jmp jmp jmp jmp jmp jmp jm...
17741,VirusShare_2afb9059cad5217d36248769ad459190,Vobfus,2013-06-04 05:17:37,jmp jmp jmp jmp jmp jmp jmp jmp jmp jmp jmp jm...
17742,VirusShare_11cfae0a335648863e395cf04dcf9390,Zbot,2013-06-04 07:33:52,push mov sub push mov cmpl je mov add mov mov ...


In [ ]:
df.to_csv('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/all_families_w_opcodes.csv')

# get time range for families

In [ ]:
# get time range for each family
df.groupby('name').agg({'date': ['min', 'max']})

date                     
                             min                  max
name                                                 
Beebone      2001-01-24 23:09:15  2013-06-04 02:45:23
Diplugem     2010-11-18 16:27:35  2013-03-12 08:51:45
Obfuscator   2001-02-19 15:35:30  2013-06-13 13:28:09
OnLineGames  2004-05-01 13:23:21  2013-04-30 12:54:35
Startpage    2003-05-15 08:30:58  2012-11-27 15:29:08
Systex.A     2012-07-12 02:56:49  2012-07-17 09:44:41
Vobfus       2001-01-07 19:54:42  2013-06-04 05:17:37
Vundo        2001-07-27 06:57:15  2013-06-02 01:02:06
Winwebsec    2001-01-27 05:35:22  2013-06-03 19:03:23
Zbot         2001-01-10 02:21:04  2013-06-04 07:33:52

# get opcode col df

In [ ]:
df_opcols = pd.read_csv('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/31_opcode_columns_frequency.csv')

In [ ]:
df_opcols

,name,add,mov,push,call,pop,lea,inc,or,cmp,...,out,jne,in,ret,int3,imul,cmc,insb,nop,other
0,Vobfus,4758,3874,8378,4836,1045,4751,2493,176,163,...,140,108,149,233,29,98,85,42,44,5590
1,Vobfus,8613,4934,10942,7936,833,6511,1984,304,203,...,143,146,179,242,54,173,59,99,34,6676
2,Zbot,17,108,33,10,22,28,6,1,2,...,0,8,0,7,0,0,0,0,0,17
3,Zbot,20,123,52,29,29,30,6,0,3,...,0,7,0,11,14,0,0,0,0,21
4,Zbot,19,139,66,27,30,19,3,0,8,...,0,9,0,11,0,0,0,0,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17739,Beebone,10654,645,943,76,386,5,1165,1256,287,...,153,91,87,131,27,288,147,346,40,4511
17740,Beebone,10638,659,931,75,387,7,1162,1247,288,...,156,97,89,134,27,278,149,334,43,4498
17741,Vobfus,8636,4370,10690,4441,614,7321,932,872,265,...,50,327,109,219,37,221,9,294,165,6019
17742,Zbot,48627,10330,4847,307,3127,179,6403,1969,5083,...,988,382,900,944,531,569,314,272,218,34797


# make opcode columns (DON'T RUN)

convert text column into opcode columns

In [ ]:
vectorizer = CountVectorizer()
vectorize = vectorizer.fit_transform(df['text'])

opcode_df = pd.DataFrame(vectorize.toarray(), columns = vectorizer.get_feature_names_out())
df_opcols = pd.concat([df, opcode_df], axis = 1)

drop filename and text columns

In [ ]:
df_opcols.drop('file_name', axis = 1, inplace = True)
df_opcols.drop('text', axis = 1, inplace = True)

In [ ]:
df_opcols = df_opcols.sort_values(by='date', ascending=True)

In [ ]:
df_opcols

,name,date,287,8087,aaa,aad,aam,aas,adc,adcb,...,xrstors,xsave,xsavec,xsaveopt,xsaves,xsetbv,xsha1,xsha256,xstore,xtest
17743,Vobfus,2001-01-07 19:54:42,0,0,22,40,33,66,182,7,...,0,0,0,0,0,0,0,0,0,0
17742,Vobfus,2001-01-09 20:15:00,0,0,39,17,29,77,146,3,...,0,0,0,0,0,0,0,0,0,0
17741,Zbot,2001-01-10 02:21:04,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17740,Zbot,2001-01-10 03:03:21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17739,Zbot,2001-01-10 03:10:13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,Beebone,2013-06-04 02:45:23,0,0,61,3,12,10,844,4,...,0,0,0,0,0,0,0,0,0,0
4,Beebone,2013-06-04 02:45:23,0,0,56,3,11,8,845,4,...,0,0,0,0,0,0,0,0,0,0
2,Vobfus,2013-06-04 05:17:37,0,0,50,6,33,14,877,3,...,0,0,0,0,0,0,0,0,0,0
1,Zbot,2013-06-04 07:33:52,1,1,707,259,211,313,1835,45,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_opcols.to_csv('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/all_families_w_opcode_columns.csv')

# non-adaptive

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_opcols.drop(['name', 'date'], axis = 1), df_opcols['name'], test_size=0.2, random_state=8)
rfc = RandomForestClassifier(n_estimators=100, random_state=8)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
skm.accuracy_score(y_test, y_pred)

0.9537897999436461

# adaptive classifiers

in chronological order

In [ ]:
arfc = forest.ARFClassifier(seed=8, leaf_prediction="mc")
st = stream.iter_pandas(df_opcols.drop(['name'], axis = 1), df_opcols['name'])
evaluate.progressive_val_score(st, arfc, metric = metrics.Accuracy(), print_every = 1000)

[1,000] Accuracy: 51.65%
[2,000] Accuracy: 64.53%
[3,000] Accuracy: 67.42%
[4,000] Accuracy: 63.34%
[5,000] Accuracy: 62.87%
[6,000] Accuracy: 64.16%
[7,000] Accuracy: 66.64%
[8,000] Accuracy: 68.58%
[9,000] Accuracy: 69.96%
[10,000] Accuracy: 69.97%
[11,000] Accuracy: 69.86%
[12,000] Accuracy: 72.18%
[13,000] Accuracy: 70.55%
[14,000] Accuracy: 70.45%
[15,000] Accuracy: 71.18%
[16,000] Accuracy: 72.98%
[17,000] Accuracy: 74.57%
[17,744] Accuracy: 74.28%


Accuracy: 74.28%

w/ shuffling

In [ ]:
arfc2 = forest.ARFClassifier(seed=8, leaf_prediction="mc")
df_shuffled = df_opcols.sample(frac=1, random_state=8)
st2 = stream.iter_pandas(df_shuffled.drop(['name'], axis = 1), df_shuffled['name'])
evaluate.progressive_val_score(st2, arfc2, metric = metrics.Accuracy(), print_every = 1000, delay = 1)

[1,000] Accuracy: 35.54%
[2,000] Accuracy: 31.07%
[3,000] Accuracy: 35.88%
[4,000] Accuracy: 38.51%
[5,000] Accuracy: 39.79%
[6,000] Accuracy: 40.56%
[7,000] Accuracy: 41.36%
[8,000] Accuracy: 42.36%
[9,000] Accuracy: 44.04%
[10,000] Accuracy: 45.84%
[11,000] Accuracy: 47.19%
[12,000] Accuracy: 47.95%
[13,000] Accuracy: 48.48%
[14,000] Accuracy: 49.56%
[15,000] Accuracy: 50.26%
[16,000] Accuracy: 50.92%
[17,000] Accuracy: 51.73%
[17,744] Accuracy: 51.90%


Accuracy: 51.90%

In [ ]:
arfc.n_drifts_detected()    # chronological

408

In [ ]:
arfc2.n_drifts_detected()   # shuffled

88

ADWIN detected 400 evolution points in the chronological data, while it detected 74 evolution points in the randomized data. This likely signifies that ADWIN is a little too sensitive (since it theoretically should not evolve when the data is randomized), but it does successfully detect evolution. This evolution detection proved to work as well: the accuracy increased almost 40%

In [ ]:
# test no drift detector, but still chronological
arfc3 = forest.ARFClassifier(seed=8, leaf_prediction="mc", drift_detector = None)
st = stream.iter_pandas(df_opcols.drop(['name'], axis = 1), df_opcols['name'])
evaluate.progressive_val_score(st, arfc3, metric = metrics.Accuracy(), print_every = 1000, delay = 1)

[1,000] Accuracy: 51.65%
[2,000] Accuracy: 64.53%
[3,000] Accuracy: 67.42%
[4,000] Accuracy: 63.34%
[5,000] Accuracy: 62.87%
[6,000] Accuracy: 64.16%
[7,000] Accuracy: 66.64%
[8,000] Accuracy: 68.58%
[9,000] Accuracy: 69.96%
[10,000] Accuracy: 69.97%
[11,000] Accuracy: 69.86%
[12,000] Accuracy: 72.18%
[13,000] Accuracy: 70.55%
[14,000] Accuracy: 70.45%


In [ ]:
arfc3.n_drifts_detected()

When testing w/o drift detection, the accuracy increased slightly. This likely indicates that there is potential creating better evolution detection systems! However, this slight change in accuracy could just be due to chance: more info needs to be gathered

In [ ]:
knnadwin = neighbors.KNNClassifier()
st = stream.iter_pandas(df_opcols.drop(['name', 'date'], axis = 1), df_opcols['name'])
evaluate.progressive_val_score(st, knnadwin, metric = metrics.Accuracy(), print_every = 1000, delay = 1)

[1,000] Accuracy: 84.28%
[2,000] Accuracy: 87.34%
[3,000] Accuracy: 84.56%
[4,000] Accuracy: 82.20%
[5,000] Accuracy: 82.22%
[6,000] Accuracy: 82.55%
[7,000] Accuracy: 83.53%
[8,000] Accuracy: 83.75%
[9,000] Accuracy: 84.13%
[10,000] Accuracy: 84.47%
[11,000] Accuracy: 84.52%
[12,000] Accuracy: 85.69%
[13,000] Accuracy: 85.46%
[14,000] Accuracy: 85.91%
[15,000] Accuracy: 86.35%
[16,000] Accuracy: 87.17%
[17,000] Accuracy: 87.92%
[17,744] Accuracy: 87.44%


Accuracy: 87.44%

In [ ]:
adb = ensemble.ADWINBoostingClassifier(model = tree.HoeffdingTreeClassifier())
st = stream.iter_pandas(df_opcols.drop(['name', 'date'], axis = 1), df_opcols['name'])
evaluate.progressive_val_score(st, adb, metric = metrics.Accuracy(), print_every = 500)

# try preprocessing w/ HMM2Vec first

## load df_hmm2vec

In [ ]:
df_hmm2vec = pd.read_csv('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/all_families_hmm2vec.csv')

## generate df_hmm2vec (DON'T RUN)

In [ ]:
##### DON'T RUN #######
# get top 30 opcodes
top_30 = opcode_df.sum(axis=0).sort_values(ascending=False).head(30).index.tolist()
enumerated = dict(zip(top_30, range(30)))

In [ ]:
enumerated = {'add': 0,'mov': 1,'push': 2,'call': 3,'pop': 4,'lea': 5,'inc': 6,'or': 7, 'cmp': 8, 'xor': 9, 'dec': 10,
              'sub': 11,'adc': 12, 'and': 13, 'movl': 14, 'xchg': 15, 'sbb': 16, 'jmp': 17, 'test': 18, 'je': 19, 'pushl': 20,
              'out': 21, 'jne': 22, 'in': 23, 'ret': 24, 'int3': 25,'imul': 26, 'cmc': 27, 'insb': 28, 'nop': 29}

In [ ]:
df_hmm2vec = pd.DataFrame(columns=range(0, 62))

start_date = datetime.date(2011,1,1)
stop_date = (start_date + pd.offsets.DateOffset(months=1)).date()
first_emission = None

for i in range(len(df)):
  # convert opcode data into numbers
  enumerated_opcodes = []
  for opcode in str(df.iloc[i]['text']).split():
    enumerated_opcodes.append(enumerated.get(opcode, 30))

  # reshape for hmm
  enumerated_opcodes = np.array(enumerated_opcodes).reshape(-1, 1)

  # make hmm model
  hmm = CategoricalHMM(n_components = 2, n_features = 31)
  hmm.fit(enumerated_opcodes)

  # get B matrix/emission
  emission = hmm.emissionprob_

  # compare w/ first emission matrix and see which order of hidden layers makes more sense
  concat_emission = np.concatenate(emission)
  concat_emission_flipped = np.concatenate([emission[1], emission[0]])
  if first_emission is None:
    first_emission = concat_emission
    df_hmm2vec.loc[int(len(df_hmm2vec.index))] = concat_emission
  elif euclidean(concat_emission, first_emission) < euclidean(concat_emission_flipped, first_emission):
    df_hmm2vec.loc[int(len(df_hmm2vec.index))] = concat_emission
  else:
    df_hmm2vec.loc[int(len(df_hmm2vec.index))] = concat_emission_flipped

  print(df_hmm2vec.shape[0])

df_hmm2vec['name'] = df['name']

In [ ]:
df_hmm2vec

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,name
0,0.114063,0.097594,0.280848,0.140894,0.022242,0.000326,0.045966,2.851340e-03,3.977668e-03,0.001469,...,0.000670,5.957778e-03,5.846959e-03,8.069436e-04,2.147467e-03,2.104698e-03,1.621459e-03,9.641227e-04,1.096171e-01,Vobfus
1,0.006216,0.121194,0.309217,0.239901,0.000293,0.197095,0.000005,2.326818e-07,6.388559e-07,0.000004,...,0.005997,7.353090e-03,4.319718e-03,2.213273e-03,7.107208e-03,2.423878e-03,4.067116e-03,1.396753e-03,2.569333e-01,Vobfus
2,0.044846,0.499800,0.075085,0.042603,0.025984,0.007429,0.009827,4.990209e-03,1.129938e-02,0.011731,...,0.043355,0.000000e+00,9.190537e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.700844e-02,Zbot
3,0.035593,0.354752,0.165913,0.092378,0.042177,0.058926,0.003735,0.000000e+00,3.373392e-03,0.022822,...,0.002865,0.000000e+00,2.426322e-02,3.104363e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.751377e-02,Zbot
4,0.038751,0.381747,0.182939,0.067010,0.085963,0.038638,0.000002,0.000000e+00,4.595598e-06,0.025054,...,0.083117,0.000000e+00,2.816340e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.838166e-01,Zbot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17739,0.025519,0.046760,0.121557,0.003636,0.003742,0.000478,0.177932,5.335157e-02,4.669847e-03,0.032486,...,0.003540,3.265600e-03,4.797759e-03,1.010516e-03,1.144837e-02,3.985850e-03,1.163734e-02,1.679926e-03,1.784533e-01,Beebone
17740,0.060985,0.017370,0.068409,0.005560,0.011844,0.000017,0.010403,4.579624e-02,2.145832e-02,0.054589,...,0.000008,3.705188e-03,7.941189e-03,2.988560e-04,1.217091e-03,8.818820e-03,2.345374e-04,1.327359e-03,4.074642e-02,Beebone
17741,0.124475,0.093195,0.398240,0.019771,0.008598,0.000008,0.035644,1.710977e-02,4.165483e-03,0.008553,...,0.005374,2.944092e-03,3.102266e-03,2.803867e-04,4.109677e-03,2.200857e-04,5.219948e-03,7.379962e-05,1.137199e-01,Vobfus
17742,0.020460,0.103727,0.048670,0.003083,0.031399,0.001797,0.064295,1.977136e-02,5.104004e-02,0.044081,...,0.000002,3.738468e-15,6.662217e-14,1.254751e-17,1.396791e-13,3.040417e-15,1.485503e-16,1.437245e-14,3.104452e-10,Zbot


In [ ]:
df_hmm2vec.to_csv('/content/drive/MyDrive/23-24/summer 24/adaptive learning research/all_families_hmm2vec.csv')

## machine learning

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_hmm2vec.drop(['name'], axis = 1), df_hmm2vec['name'], test_size=0.2, random_state=8)

In [ ]:
rfc_h2v = RandomForestClassifier(n_estimators=100, random_state=8)
rfc_h2v.fit(X_train, y_train)
y_pred = rfc_h2v.predict(X_test)
skm.accuracy_score(y_test, y_pred)

0.8675683291067906

## adaptive learning

In [ ]:
adwin = drift.ADWIN()
st = stream.iter_pandas(df_hmm2vec.drop(['name'], axis = 1), df_hmm2vec['name'])
for i, (X, y) in enumerate(st):
  adwin.update(y)
  if adwin.drift_detected:
    print(f"Change detected at index {i}, input value: {y}")

In [ ]:
arfc_h2v = forest.ARFClassifier(seed=8, leaf_prediction="mc")
st = stream.iter_pandas(df_hmm2vec.drop(['name'], axis = 1), df_hmm2vec['name'])
evaluate.progressive_val_score(st, arfc_h2v, metric = metrics.Accuracy(), print_every = 1000, delay = 1)

[1,000] Accuracy: 53.15%
[2,000] Accuracy: 65.63%
[3,000] Accuracy: 68.06%
[4,000] Accuracy: 63.82%
[5,000] Accuracy: 61.87%
[6,000] Accuracy: 63.38%
[7,000] Accuracy: 65.61%
[8,000] Accuracy: 66.72%
[9,000] Accuracy: 68.06%
[10,000] Accuracy: 67.66%
[11,000] Accuracy: 66.94%
[12,000] Accuracy: 69.26%
[13,000] Accuracy: 68.27%
[14,000] Accuracy: 67.73%
[15,000] Accuracy: 68.74%
[16,000] Accuracy: 70.70%
[17,000] Accuracy: 72.42%
[17,744] Accuracy: 72.11%


Accuracy: 72.11%

In [ ]:
arfc_h2v.n_drifts_detected()

360

In [ ]:
knnadwin_h2v = neighbors.KNNClassifier()
st = stream.iter_pandas(df_hmm2vec.drop(['name'], axis = 1), df_hmm2vec['name'])
evaluate.progressive_val_score(st, knnadwin_h2v, metric = metrics.Accuracy(), print_every = 1000, delay = 1)

[1,000] Accuracy: 64.86%
[2,000] Accuracy: 71.54%
[3,000] Accuracy: 70.92%
[4,000] Accuracy: 67.72%
[5,000] Accuracy: 68.15%
[6,000] Accuracy: 70.41%
[7,000] Accuracy: 71.31%
[8,000] Accuracy: 72.65%
[9,000] Accuracy: 74.12%
[10,000] Accuracy: 74.57%
[11,000] Accuracy: 74.57%
[12,000] Accuracy: 75.97%
[13,000] Accuracy: 75.42%
[14,000] Accuracy: 74.87%
[15,000] Accuracy: 75.22%
[16,000] Accuracy: 76.70%
[17,000] Accuracy: 78.08%
[17,744] Accuracy: 77.80%


Accuracy: 77.80%

In [ ]:
adb_h2v = ensemble.ADWINBoostingClassifier(model = tree.HoeffdingTreeClassifier())
st = stream.iter_pandas(df_hmm2vec.drop(['name'], axis = 1), df_hmm2vec['name'])
evaluate.progressive_val_score(st, adb_h2v, metric = metrics.Accuracy(), print_every = 500)

[500] Accuracy: 67.74%
[1,000] Accuracy: 52.55%
[1,500] Accuracy: 55.37%
[2,000] Accuracy: 66.33%
[2,500] Accuracy: 72.79%
[3,000] Accuracy: 68.09%
[3,500] Accuracy: 65.65%
[4,000] Accuracy: 61.52%
[4,500] Accuracy: 60.70%
[5,000] Accuracy: 60.93%
[5,500] Accuracy: 62.90%
[6,000] Accuracy: 63.14%
[6,500] Accuracy: 63.39%
[7,000] Accuracy: 62.65%
[7,500] Accuracy: 63.50%
[8,000] Accuracy: 64.48%
[8,500] Accuracy: 66.02%
[9,000] Accuracy: 66.80%
[9,500] Accuracy: 67.72%
[10,000] Accuracy: 67.84%
[10,500] Accuracy: 67.93%
[11,000] Accuracy: 67.62%
[11,500] Accuracy: 66.58%
[12,000] Accuracy: 67.96%
[12,500] Accuracy: 67.35%
[13,000] Accuracy: 67.35%
[13,500] Accuracy: 67.19%
[14,000] Accuracy: 67.07%
[14,500] Accuracy: 67.94%
[15,000] Accuracy: 68.27%
[15,500] Accuracy: 69.26%
[16,000] Accuracy: 70.22%
[16,500] Accuracy: 71.12%
[17,000] Accuracy: 71.97%
[17,500] Accuracy: 72.10%
[17,744] Accuracy: 72.15%


Accuracy: 72.15%

In [ ]:
arfc_h2v2 = forest.ARFClassifier(seed=8, leaf_prediction="mc")
st = stream.iter_pandas(df_hmm2vec.drop(['name'], axis = 1), df_hmm2vec['name'])
evaluate.progressive_val_score(st, arfc_h2v2, metric = [metrics.Accuracy(), metrics.Precision(), metrics.Recall(), metrics.F1()], print_every = 1000, delay = 1)

# try preprocessing w/ word2vec first

In [ ]:
enumerated = {'add': 0,'mov': 1,'push': 2,'call': 3,'pop': 4,'lea': 5,'inc': 6,'or': 7, 'cmp': 8, 'xor': 9, 'dec': 10,
              'sub': 11,'adc': 12, 'and': 13, 'movl': 14, 'xchg': 15, 'sbb': 16, 'jmp': 17, 'test': 18, 'je': 19, 'pushl': 20,
              'out': 21, 'jne': 22, 'in': 23, 'ret': 24, 'int3': 25,'imul': 26, 'cmc': 27, 'insb': 28, 'nop': 29}

In [ ]:
df_w2v = pd.DataFrame(columns=range(0, 30))

first_concat = None

for i in range(len(df)):
  #make w2v model
  texts = [str(df.iloc[i]['text']).split()]
  w2v = Word2Vec(texts, vector_size = 2, window = 5)
  keyed_vecs = w2v.wv

  # get embeddings
  embeddings = []
  for opcode in enumerated:
    if opcode in keyed_vecs:
      embeddings.append(keyed_vecs[opcode])
    else:
      embeddings.append(np.zeros(2))

  # compare w/ first embedding vector and see which order of dimensions makes more sense
  concat_embed = np.concatenate(embeddings)
  concat_embed_flipped = np.concatenate([embed[1], embed[0]] for embed in embeddings)
  print(concat_embed[0:6])
  print(concat_embed[0:6])
  if first_concat is None:
    first_concat = concat_embed
    df_w2v.loc[int(len(df_w2v.index))] = concat_embed
  elif euclidean(concat_embed, first_concat) < euclidean(concat_embed_flipped, first_concat):
    df_w2v.loc[int(len(df_w2v.index))] = concat_embed
  else:
    df_w2v.loc[int(len(df_w2v.index))] = concat_embed_flipped

  print(i)

df_w2v['name'] = df['name']

# hmm2vec + frequencies

In [ ]:
df_h2v_freq = pd.merge(df_hmm2vec, df_opcols, left_index = True, right_index = True)

In [ ]:
df_h2v_freq.columns[62:70]

Index(['name_x', 'name_y', 'date', '287', '8087', 'aaa', 'aad', 'aam'], dtype='object')